In [1]:
import os
from PIL import Image
import torch
import cv2
from torch.utils.data import Dataset, DataLoader

In [20]:
class PIVDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.flow_types = os.listdir(root_dir)
        self.num_samples = self.calculate_num_samples()
    
    def calculate_num_samples(self):
        num_samples = 0
        for flow_type in self.flow_types:
            flow_dir = os.path.join(self.root_dir, flow_type)
            file_names = [f for f in os.listdir(flow_dir) if f.endswith("_img1.tif")]
            num_samples += len(file_names)
        print(num_samples)
        return num_samples

    def __len__(self):
        return self.num_samples

    def load_velocity_field(flow_path):
        # Use OpenCV to read the .flo file
        flow = cv2.readOpticalFlow(flow_path)
        # Convert the loaded flow data to a NumPy array
        flow = flow[..., 0:2]  # Extract the U and V components
        return flow
       

    def __getitem__(self, idx):
    # Initialize the flow_type_idx and remaining_idx
        flow_type_idx = 0
        remaining_idx = idx
        
        # Iterate through flow types to find the correct one
        while flow_type_idx < len(self.flow_types):
            num_samples = len(os.listdir(os.path.join(self.root_dir, self.flow_types[flow_type_idx])))
            if (remaining_idx < num_samples) :
                break  # Found the correct flow type
            else:
                remaining_idx -= num_samples  # Adjust the remaining index
                flow_type_idx += 1
        
        # Now that we have the correct flow type, proceed to load the data
        flow_type = self.flow_types[flow_type_idx]
        flow_dir = os.path.join(self.root_dir, flow_type)
        file_names = [f for f in os.listdir(flow_dir) if f.endswith("_img1.tif")]
        
        instance_name = file_names[remaining_idx]  # Use remaining_idx to access the file
        img1_path = os.path.join(flow_dir, instance_name)
        img2_path = os.path.join(flow_dir, instance_name.replace("_img1.tif", "_img2.tif"))
        flow_path = os.path.join(flow_dir, instance_name.replace("_img1.tif", "_flow.flo"))
        
        img1 = Image.open(img1_path)
        img2 = Image.open(img2_path)
        flow = self.load_velocity_field(flow_path)  # Implement a function to load .flo files
        
        return {"image_pair": (img1, img2), "velocity_field": flow}


 # Create an instance of the custom dataset
dataset = PIVDataset(root_dir='C:/Users/estal/OneDrive - ROCKWOOL Group/Documents/GD/Thesis/04_Code/src/data/raw/PIV_dataset/PIV-genImages/data')

# Create a data loader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
print(len(dataset))
print("success!")

8250
8250
success!


In [21]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have a data loader called 'data_loader' from your PIVDataset
for batch in data_loader:
    image_pairs = batch["image_pair"]  # This should contain a tuple of (img1, img2)
    velocity_fields = batch["velocity_field"]

    # Process the first pair in the first batch and then break
    img1 = image_pairs[0][0]  # First image in the pair
    img2 = image_pairs[0][1]  # Second image in the pair
    flow = velocity_fields[0]  # Velocity field

    # Convert the PyTorch tensor to a NumPy array for visualization
    img1 = img1.numpy().transpose((1, 2, 0))
    img2 = img2.numpy().transpose((1, 2, 0))
    flow = flow.numpy()

    # Visualization of the first image pair and velocity field
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(img1)
    plt.title('Image 1')
    
    plt.subplot(1, 3, 2)
    plt.imshow(img2)
    plt.title('Image 2')
    
    plt.subplot(1, 3, 3)
    plt.imshow(np.sqrt(flow[..., 0]**2 + flow[..., 1]**2), cmap='viridis')
    plt.title('Velocity Magnitude')

    plt.show()

    break  # Exit the loop after processing the first pair



TypeError: PIVDataset.load_velocity_field() takes 1 positional argument but 2 were given